# ECEN743 Spring 2025 - Assignment 2
## Tabular RL Algorithms

In this assignment, you will solve the FrozenLake-v1 environment from [Gymnasium](https://gymnasium.farama.org/). You will be using this helper file to answer questions in your assignment. 

**Note that you do not need to start from the scratch. Only write your code between the following lines. Do not modify other parts.**  
"### YOUR CODE HERE"  
"### END OF YOUR CODE"

## Introduction of the FrozenLake Environment

Frozen lake involves crossing a frozen lake from start to goal without falling into any holes by walking over the frozen lake. The player may not always move in the intended direction due to the slippery nature of the frozen lake. The game starts with the player at location [0,0] of the frozen lake grid world with the goal located at far extent of the world e.g. [3,3] for the 4x4 environment. Holes in the ice are distributed in set locations using a pre-determined map, and the player makes moves until they reach the goal or fall in a hole. The map is given below for your reference

        SFFF
        FHFH
        FFFH
        HFFG
    S : starting point, safe
    F : frozen surface, safe
    H : hole, fall to your doom
    G : goal, where the frisbee is located
    
    
### Action Space
The player/agent can take 4 discrete actions, in the range {0,3}
* 0: Move left
* 1: Move down
* 2: Move right 
* 4: Move up


### State Space
The environment consists of 16 states. The state is a value representing the player’s current position as current_row * nrows + current_col (where both the row and col start at 0).
For example, the goal position in the 4x4 map can be calculated as follows: 3 * 4 + 3 = 15.


### Starting State
The episode starts with the player in state [0] (location [0, 0]).


### Rewards 

* Reach goal: +1
* Reach hole: 0
* Reach frozen: 0

### Episode End
The episode ends if the following happens:
#### 1.Termination:
* The player moves into a hole.
* The player reaches the goal at max(nrow) * max(ncol) - 1 (location [max(nrow)-1, max(ncol)-1]).

#### 2.Truncation:
* The length of the episode is 100 for 4x4 environment.

For more info refer to source: https://gymnasium.farama.org/environments/toy_text/frozen_lake/

### The Environment Parameters
* Use discount factor, $\gamma = 0.9$
* The environment is slippery, ie., the transition kernel is stochastic.
* The transition kernel P is a dictionary. 
* P[state][action] is tuples with (probability, nextstate, reward, terminal)

**Run the following initializer. Make sure you can execute it without any error.**

If you wish to finish this assignment using Google Colab. Uncomment the following commands and run them.

In [ ]:
# !pip install swig
# !pip install gymnasium[box2d]

In [1]:
import gymnasium as gym
%matplotlib inline
import seaborn
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import numpy as np
np.set_printoptions(precision=4, suppress=True)

env = gym.make('FrozenLake-v1', desc=None,map_name="4x4", is_slippery=True)
while hasattr(env, 'env'):
    env = env.env
gamma = 0.9


# The optimal Q-value function from HW01
Q_opt = np.array([[0.06887237, 0.06663045, 0.06663045, 0.05974078],
                 [0.03907988, 0.04297989, 0.04073797, 0.06139887],
                 [0.07439831, 0.06881719, 0.0727172,  0.05747583],
                 [0.03905686, 0.03905686, 0.03347574, 0.05579473],
                 [0.09183769, 0.07117679, 0.06428712, 0.04821147],
                 [0.,         0.,         0.,         0.        ],
                 [0.11220205, 0.08988305, 0.11220205, 0.02231899],
                 [0.,         0.,         0.,         0.        ],
                 [0.07117679, 0.11787214, 0.1017965,  0.14542271],
                 [0.15760471, 0.24748776, 0.20386154, 0.13350927],
                 [0.29961112, 0.26595078, 0.22536519, 0.10790627],
                 [0.,         0.,         0.,         0.        ],
                 [0.,         0.,         0.,         0.        ],
                 [0.18822442, 0.30568334, 0.37992926, 0.26595078],
                 [0.39556639, 0.63901667, 0.61492124, 0.53719488],
                 [0.,         0.,         0.,         0.        ]])

## 1. Tabular Q-Learning

### The decay of learning rate and exploration factor
We aim to learn the optimal Q-value function through online exploration. Intuitively, after sufficient exploration, our learner should already have good amount of awareness in the scary FrozenLake. Too much exploration results in a sub-optimal policy. Hence, we should dynamically adjust our exploration scheme.

The convergence of an online learning algorithm is highly correlated to the stochasticity of the environment. As a result, you may see your Q-value function fluctuates frequently. This could be because the learner just faced a bad episode! However, after training for a while, your learner should have seen sufficient number of both good and bad episodes. Decaying the learning rate can help the algorithm to converge to the optimal policy.

### Your Task
1. Implement `lr_decay` and `epsilon_decay`. Experiment with linear decay, exponential decay, etc. You do not need to find **THE** best decay scheme.
2. Implement `epsilon_greedy'. One way to do this is to sample a Bernoulli random variable with parameter $\epsilon$. Note that, $\epsilon$ is the probability that **you choose a random action instead of the greedy action**.
3. Implement Q-Learning update.

In [ ]:
def lr_decay(k):
    """Return the learning rate for k-th episode

    Args:
        k: The index of episode.
    Returns:
        lr: The learning rate for this episode.
    """
    ### YOUR CODE HERE
    # TASK 1
    lr = 0.5
    ### END OF YOUR CODE
    return lr

def epsilon_decay(k):
    """Return the decayed exploration factor
    
    Args:
        k: The index of episode.
    Returns:
        epsilon: The exploration factor for this episode.
    """    
    ### YOUR CODE HERE
    # TASK 1
    epsilon = 0.5
    ### END OF YOUR CODE
    return epsilon

def epsilon_greedy(Q, s, epsilon):
    """Outputs an action acoording to epsilon-greedy policy

    Args:
        Q (matrix of |S|x|A|): matrix of Q-functions, where Q(s,a) is at s-th row and a-th column
        s (int): current state where next action is inquired upon
        epsilon (0<=float<=1): Pr(random exploration and avoid optimal action) = epsilon

    Return:
        a (int): next action
    """
    ### YOUR CODE HERE
    # TASK 2
    a = 0
    ### END OF YOUR CODE
    return int(a)

In [ ]:
no_of_actions = env.action_space.n
no_of_states = env.observation_space.n

max_step = 100
max_episode = 30000

Q = np.zeros((no_of_states, no_of_actions))
G_list = [] # cumulative discounted reward in episode k
convergence_list = [] # record ||Q_k-Q_STAR||_{2}

for k in range(max_episode):
    s0, _ = env.reset()  # start new episode
    path_k = [s0]  # initial state for new episode
    G_k = 0  # reset cumulative discounted reward for episode k
    for t in range(max_step):
        s_t = path_k[-1] # observe current state s_t
        a_t = epsilon_greedy(Q, s_t, epsilon_decay(k))  # decide what action to take at step t
        ns_t, r_t, terminal, _, _ = env.step(a_t)  # take a step
        ### YOUR CODE HERE
        # TASK 3
        # Hint: you should call lr_decay somewhere here.
        # Q[s_t][a_t] = 
        ### END OF YOUR CODE
        # We have updated the following line
        G_k += r_t  # update cumulative reward
        path_k.append(ns_t)  # record next state
        if terminal:  # next state is a terminal state, episode ended
            break
    convergence_list.append(np.linalg.norm(Q.flatten()-Q_opt.flatten())) # calc and store Euclidean norm of Q_k-Q*
    G_list.append(G_k) # record cumulative discounted reward for episode k

### 1a. Plot $G_k$ using a sliding window

$G_k$ is the cumulative reward obtained in episode $k$.

In [ ]:
### YOUR CODE HERE
# Some plot stuff
### END OF YOUR CODE

### 1b. Plot $||Q_k - Q^*||$

Note that use `Q_opt` for $Q^*$. It is given to you in the previous section of this file. You do not need to copy it from HW1.

In [ ]:
### YOUR CODE HERE
# Some plot stuff
### END OF YOUR CODE

### 1c. Question

What is the policy and Q-value function obtained at the end of the learning? Are you
able to learn the optimal policy? That is, how to get the optimal policy from the optimal Q-value function?

In [ ]:
### YOUR CODE HERE
print(Q)
print(Q_opt)
# Compute the final policy
### END OF YOUR CODE

**Write your answer below.**  
Answer:

## 2. Behavior Policy  

Implement tabular Q-learning with a uniformly random policy (where
each action is taken with equal probability) as the behavior policy. Compare the convergence
with the ε-greedy exploration approach. Explain your observations and inference. Can you
implement a better behavior policy and show its effectiveness?

In [ ]:
### YOUR CODE HERE
# Freestyle!
### END OF YOUR CODE

**Write your answer below.**  
Answer:

## 3. TD-Learning

Consider the following polices: (i) the optimal policy obtained from QVI,
and (ii) a uniformly random policy where each action is taken with equal probability. Learn
the value of the these polices using:

### 3a. Monte Carlo (MC) Learning

In [ ]:
### YOUR CODE HERE
# Freestyle!
### END OF YOUR CODE

### 3b. Temporal Difference (TD) Learning

In [ ]:
### YOUR CODE HERE
# Freestyle!
### END OF YOUR CODE

### 3c. What are the trade-offs of between MC vs TD?

**Write your answer below.**  
Answer: